# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [1]:
!docker run -d --name nginx --network=none -p 81:81 nginx

docker: Error response from daemon: Conflict. The container name "/nginx" is already in use by container "cf330b5a960989db50e0992501c3560af246f9fb74ffe31919c4c6a1317fb977". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [2]:
!curl localhost

#the curl command should fail on our system 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2223 ms: Could not connect to server


Lets run it on the docker container 

In [6]:
!docker exec -it nginx /bin/sh -c "curl google.com"



the input device is not a TTY.  If you are using mintty, try prefixing the command with 'winpty'


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [7]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [8]:
!docker network ls 

NETWORK ID     NAME      DRIVER    SCOPE
c0765276b61e   bridge    bridge    local
9eeb3540284c   host      host      local
dd01af6a058c   none      null      local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [9]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "dd01af6a058c6deb031ff4e097224a33636a514f16f9d129610a0544f52b877b",
        "Created": "2025-04-19T03:13:40.59360815Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "cf330b5a960989db50e0992501c3560af246f9fb74ffe31919c4c6a1317fb977": {
                "Name": "nginx",
                "EndpointID": "0573851af97679fd7e35f9ec775535d42eb6e415b018757d59028ec23b050913",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [10]:
!pip install docker

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


Docker version: 28.0.4


Lets create a quick code to list the containers using the docker client

In [12]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

nginx - running


Here you can see the running container and no surprise it is the nginx we started before


In [14]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [15]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

2b5d0afd6b6d00def37dbd46511fdf9fb923be8a4369c0f81904356d6082a798


Unable to find image 'mysql:8' locally
8: Pulling from library/mysql
07e8f079852f: Pulling fs layer
cea172a6e83b: Pulling fs layer
6c228f7bea3b: Pulling fs layer
290ec4c227a4: Pulling fs layer
182039f5bee3: Pulling fs layer
b3dd4e5cd318: Pulling fs layer
cb416885cc1f: Pulling fs layer
aad4571eacc0: Pulling fs layer
10eaa68c5921: Pulling fs layer
d1e5287da18a: Pulling fs layer
10eaa68c5921: Download complete
6c228f7bea3b: Download complete
07e8f079852f: Download complete
aad4571eacc0: Download complete
b3dd4e5cd318: Download complete
290ec4c227a4: Download complete
cb416885cc1f: Download complete
d1e5287da18a: Download complete
cea172a6e83b: Download complete
cea172a6e83b: Pull complete
6c228f7bea3b: Pull complete
290ec4c227a4: Pull complete
10eaa68c5921: Pull complete
cb416885cc1f: Pull complete
07e8f079852f: Pull complete
aad4571eacc0: Pull complete
d1e5287da18a: Pull complete
182039f5bee3: Download complete
b3dd4e5cd318: Pull complete
182039f5bee3: Pull complete
Digest: sha256:3a8925

## The following needs to be run in the terminal

In [16]:
!docker run -d --name adminer -p 8080:8080 adminer


7a0ee76020de3553d53b64a25289cd28fa1eae8f54743c7a7b5bdeb65025bedd


Unable to find image 'adminer:latest' locally
latest: Pulling from library/adminer
b1220d176a8d: Pulling fs layer
52e9abf80869: Pulling fs layer
6d95abe48138: Pulling fs layer
0d55cff53123: Pulling fs layer
a80d3af78a9d: Pulling fs layer
dc6d32971fca: Pulling fs layer
f3fe38446507: Pulling fs layer
88d591373a19: Pulling fs layer
e434f3dae19e: Pulling fs layer
8b847f2809a2: Pulling fs layer
ffeef6b41a56: Pulling fs layer
115c7a04f6c0: Pulling fs layer
c6d0aade0806: Pulling fs layer
93b26376816c: Pulling fs layer
f18232174bc9: Pulling fs layer
4f4fb700ef54: Pulling fs layer
dc6d32971fca: Download complete
0d55cff53123: Download complete
115c7a04f6c0: Download complete
b1220d176a8d: Download complete
52e9abf80869: Download complete
f3fe38446507: Download complete
8b847f2809a2: Download complete
a80d3af78a9d: Download complete
c6d0aade0806: Download complete
4f4fb700ef54: Download complete
6d95abe48138: Download complete
93b26376816c: Download complete
e434f3dae19e: Download complete
f1823

Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [17]:
!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


------------------------------
mysql in network 'bridge' has IP: 172.17.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
#!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
#print("-"*30)
show_container_ip("adminer")


adminer in network 'bridge' has IP: 172.17.0.3


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [19]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        #print(f"Waiting for {container.name} to stop...")
        #while container.status != 'exited':
           # pass
        
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


adminer - running
mysql - running
nginx - running
Removing adminer
Removing mysql
Removing nginx


## Setting up the network

In [20]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

36bf326666d3671eb38198c2e4684bd68436f0a0a7762d7e67d93faed9607e76


In [21]:
!docker network inspect yourNetwork

[
    {
        "Name": "yourNetwork",
        "Id": "36bf326666d3671eb38198c2e4684bd68436f0a0a7762d7e67d93faed9607e76",
        "Created": "2025-04-19T04:14:16.437641742Z",
        "Scope": "local",
        "Driver": "bridge",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": {},
            "Config": [
                {
                    "Subnet": "172.18.0.0/16",
                    "Gateway": "172.18.0.1"
                }
            ]
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {},
        "Options": {},
        "Labels": {}
    }
]


In [1]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


08348b8912e5f718d62827eafeebf28f796673be8d9932c440e87feff0a131fd


In [2]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

99884d982bf9e842cba90081aed7d19e1e5db2cbe7926ffef725e6391e2d5be9


In [24]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [25]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

'grep' is not recognized as an internal or external command,
operable program or batch file.


------------------------------
mysql in network 'yourNetwork' has IP: 172.18.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

            "Networks": {
                "yourNetwork": {
                    "IPAMConfig": null,
                    "IPAddress": "172.18.0.3",
PING 172.18.0.3 (172.18.0.3) 56(84) bytes of data.
64 bytes from 172.18.0.3: icmp_seq=1 ttl=64 time=0.063 ms

--- 172.18.0.3 ping statistics ---
1 packets transmitted, 1 received, 0% packet loss, time 0ms
rtt min/avg/max/mdev = 0.063/0.063/0.063/0.000 ms
------------------------------


NameError: name 'show_container_ip' is not defined

# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [26]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("mysql")
show_container_ip("adminer")

mysql in network 'yourNetwork' has IP: 172.18.0.2
adminer in network 'yourNetwork' has IP: 172.18.0.3



## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/



In [ ]:
import time
import threading
import docker

client = docker.from_env()

# List all container names and status
def list_running_containers():
    containers = client.containers.list(all=True)
    for container in containers:
        print(f"{container.name} - {container.status}")

# Monitor container status and restart if not running
def monitor_container(container_name):
    failure_count = 0
    max_failures = 3
    last_status = None

    while True:
        try:
            container = client.containers.get(container_name)
            container.reload()
            status = container.status

            if status != 'running':
                print(f"{container_name} is not running. Attempting restart...")

                container.start()
                time.sleep(10)
                container.reload()

                if container.status == 'running':
                    print(f"{container_name} restarted successfully.")
                    failure_count = 0
                else:
                    print(f"{container_name} failed to restart.")
                    failure_count += 1

                    if failure_count >= max_failures:
                        print(f"{container_name} has failed {max_failures} times. Stopping monitoring.")
                        break
            else:
                if status != last_status:
                    print(f"{container_name} is running.")
                failure_count = 0

            last_status = status

        except docker.errors.NotFound:
            print(f"{container_name} not found.")
        except Exception as e:
            print(f"Error monitoring {container_name}: {e}")

        time.sleep(30)

# Scheduled restart every 24 hours
def restart_container_daily(container_name):
    while True:
        try:
            container = client.containers.get(container_name)
            print(f"Scheduled restart for {container_name}...")
            container.restart()
        except Exception as e:
            print(f"Error restarting {container_name}: {e}")
        time.sleep(86400)  # 24 hours

# Start containers
def start_specific_containers(container_names):
    for name in container_names:
        try:
            container = client.containers.get(name)
            container.reload()
            if container.status != 'running':
                print(f"Starting container: {name}")
                container.start()
            else:
                print(f"{name} is already running.")
        except Exception as e:
            print(f"Error starting {name}: {e}")

# Stop containers
def stop_specific_containers(container_names):
    for name in container_names:
        try:
            container = client.containers.get(name)
            container.reload()
            if container.status == 'running':
                print(f"Stopping container: {name}")
                container.stop()
            else:
                print(f"{name} is already stopped.")
        except Exception as e:
            print(f"Error stopping {name}: {e}")

# Run the full setup
if __name__ == "__main__":
    containers_to_manage = ["mysql", "adminer"]

    print("=== Docker Container Monitor ===")
    list_running_containers()

    for name in containers_to_manage:
        threading.Thread(target=monitor_container, args=(name,), daemon=True).start()
        threading.Thread(target=restart_container_daily, args=(name,), daemon=True).start()

    while True:
        time.sleep(60)


=== Docker Container Monitor ===
adminer - exited
mysql - exited
Scheduled restart for mysql...
Scheduled restart for adminer...
mysql is not running. Attempting restart...
adminer is not running. Attempting restart...
mysql restarted successfully.
adminer restarted successfully.
mysql is running.
adminer is running.
